In [1]:
from pathlib import Path
import os
import pandas as pd
from datetime import datetime

In [ ]:

# Ruta y fecha
ruta = Path.cwd()
fecha = datetime.now().strftime("%Y%m%d")

# Listar archivos
archivos = os.listdir(ruta / fecha)
archvos

In [ ]:

# Leer archivos
df_totales = pd.read_csv(f'{ruta}/{fecha}/{archivos[0]}', sep=';', encoding='utf-8')
df_ingresos = pd.read_csv(f'{ruta}/{fecha}/{archivos[2]}', sep=',', encoding='utf-8')
df_eliminaciones = pd.read_csv(f'{ruta}/{fecha}/{archivos[1]}', sep=',', encoding='utf-8')

In [ ]:
# Estandarización de columnas
df_ingresos['RUTID_SOCIEDAD'] = df_ingresos['RUTID_SOCIEDAD'].astype(str).str.zfill(10)
df_ingresos['RUTID_SOCIO'] = df_ingresos['RUTID_SOCIO'].astype(str).str.zfill(10)

df_eliminaciones['RUTID_SOCIEDAD'] = df_eliminaciones['RUTID_SOCIEDAD'].astype(str).str.zfill(10)
df_eliminaciones['RUTID_SOCIO'] = df_eliminaciones['RUTID_SOCIO'].astype(str).str.zfill(10)

df_totales['rut_sociedad'] = df_totales['rut_sociedad'].astype(str).str.zfill(10)
df_totales['rut_socio'] = df_totales['rut_socio'].astype(str).str.zfill(10)

In [ ]:
# Crear columnas concat
df_totales['concat'] = df_totales['rut_sociedad'] + df_totales['rut_socio']
df_eliminaciones['concat'] = df_eliminaciones['RUTID_SOCIEDAD'] + df_eliminaciones['RUTID_SOCIO']
df_ingresos['concat'] = df_ingresos['RUTID_SOCIEDAD'] + df_ingresos['RUTID_SOCIO']

In [ ]:
# Agregar ingresos a totales
ya_existen = df_ingresos[df_ingresos['concat'].isin(df_totales['concat'])]
total_sin_ingresar = df_ingresos[~df_ingresos['concat'].isin(df_totales['concat'])]
df_totales = pd.concat([df_totales, total_sin_ingresar], ignore_index=True)

In [ ]:
# Eliminar registros en eliminaciones de totales
df_totales = df_totales[~df_totales['concat'].isin(df_eliminaciones['concat'])]

In [ ]:
# Validaciones finales
assert len(df_totales[df_totales['rut_sociedad'] == df_totales['rut_socio']]) == 0, "Error: Hay RUTs iguales en las columnas."


In [ ]:
# Exportar resultado final
df_totales.drop(columns=['concat'], inplace=True)
df_totales.to_csv(f'{ruta}/{fecha}/MALLA_SOC_{fecha}.csv', sep=',', index=False)